In [3]:
import random
import time
import csv
from pathlib import Path

def generate_question(level):
    ops = ['+','-','*','/']
    if level == 'easy':
        a, b = random.randint(1,10), random.randint(1,10)
    elif level == 'medium':
        a, b = random.randint(1,50), random.randint(1,50)
    else:
        a, b = random.randint(1,200), random.randint(1,200)
    op = random.choice(ops)
    # avoid division by zero and make division cleaner
    if op == '/':
        b = random.randint(1,10)
        a = a * b
    return a, op, b

def compute_answer(a, op, b):
    if op == '+': return a + b
    if op == '-': return a - b
    if op == '*': return a * b
    if op == '/': return a / b

def quiz(rounds=10, level='easy', timed=False):
    score = 0
    results = []
    start_time = time.time()
    for i in range(1, rounds+1):
        a, op, b = generate_question(level)
        correct = compute_answer(a, op, b)
        prompt = f"Q{i}: {a} {op} {b} = "
        if timed:
            t0 = time.time()
        try:
            ans = input(prompt)
            if ans.strip().lower() == 'q':
                print("Exiting current quiz early...")
                break
            if ans.strip() == '':
                print("Skipped.")
                results.append((prompt, None, correct, False))
                continue
            user = float(ans)
        except ValueError:
            print("Invalid. counted as wrong.")
            user = None
        #avoid floating point error
        if user is not None and abs(user - correct) < 1e-6: 
            print("Correct!")
            score += 1
            correct_flag = True
        else:
            print(f"Wrong — correct answer: {correct}")
            correct_flag = False
        if timed:
            elapsed = time.time() - t0
        else:
            elapsed = None
        results.append((prompt, user, correct, correct_flag, elapsed))
    total_time = time.time() - start_time
    print(f"\nScore: {score}/{len(results)} — Time: {total_time:.2f}s")
    save = input("Save results to CSV? (y/n): ").lower().startswith('y')
    if save:
        p = Path('quiz_results.csv')
        with p.open('a', newline='') as f:
            writer = csv.writer(f)
            for r in results:
                writer.writerow([*r])
        print(f"Results appended to {p}")
    return score, results

def main():
    print("***Arithmetic Quiz***")
    print("Type 'q' at any prompt to quit at any time.\n")

    while True:
        level = input("Choose difficulty (easy/medium/hard): ") or 'easy'
        if level.lower() == 'q':
            print("Goodbye!")
            break

        try:
            rounds = input("Number of questions: ") or "10"
            if rounds.lower() == 'q':
                print("Goodbye!")
                break
            rounds = int(rounds)
        except ValueError:
            print("Invalid number — defaulting to 10.")
            rounds = 10

        timed = input("Timed? (y/n) [n]: ").lower().startswith('y')
        quiz(rounds=rounds, level=level, timed=timed)

        again = input("\nDo you want to take another quiz? (y/n): ").lower()
        if again != 'y':
            print("Thanks for playing! Goodbye!")
            break

if __name__ == '__main__':
    main()


=== Arithmetic Quiz ===
Type 'q' at any prompt to quit at any time.



Choose difficulty (easy/medium/hard) [easy]:  easy
Number of questions [10]:  3
Timed? (y/n) [n]:  n
Q1: 9 + 8 =  17


Correct!


Q2: 5 * 10 =  50


Correct!


Q3: 7 * 1 =  7


Correct!

Score: 3/3 — Time: 6.51s


Save results to CSV? (y/n):  n

Do you want to take another quiz? (y/n):  n 


Thanks for playing! Goodbye!
